In [1]:
import pyspark
from pyspark.sql import Column, DataFrame, SparkSession, functions
from pyspark.sql.functions import *
from py4j.java_collections import MapConverter
import shutil
import random
import threading
conf = pyspark.SparkConf()
conf.setMaster("spark://spark:7077") 

conf.set("spark.hadoop.fs.s3a.endpoint", 'http://s3:9000') \
    .set("spark.hadoop.fs.s3a.access.key", 'minio') \
    .set("spark.hadoop.fs.s3a.secret.key", 'minio123') \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
sc = pyspark.SparkContext(conf=conf)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
com.databricks#dbutils-api_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0a3b496e-d15f-4766-a484-0ceb71523cde;1.0
	confs: [default]
	found com.databricks#dbutils-api_2.12;0.0.5 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession(sc).builder.appName("streaming").getOrCreate()
from delta.tables import *

In [5]:
#raw data received from kafka is stored under S3 customers
OBJECTURL_TEST = 's3a://minio-sink-bucket/topics/customers'
schema=spark.read.format('json').load(OBJECTURL_TEST).schema
streamingRawDF=spark.readStream.format("json").schema(schema).load(OBJECTURL_TEST)
stream=streamingRawDF.select(col("after.id"), col("after.first_name"), col("after.last_name"), col("after.email")).writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", 's3a://minio-sink-bucket/delta/bronze/customers/checkpoints/').start('s3a://minio-sink-bucket/delta/bronze/customers/data/')
#set it to low number in order to see the acidity
#for 5 seconds it will transfer some of the data and with fail, and the delta will not be created
stream.awaitTermination(20)
stream.stop()
deltaTable = DeltaTable.forPath(spark, 's3a://minio-sink-bucket/delta/bronze/customers/data')
print("#############  Original Delta Table ###############")
deltaTable.toDF().show()

#############  Original Delta Table ###############


+----+----------+---------+--------------------+
|  id|first_name|last_name|               email|
+----+----------+---------+--------------------+
|1001|     Sally|   Thomas|sally.thomas@acme...|
|1004|      Anne|Kretchmar|  annek@noanswer.org|
|1002|    George|   Bailey|  gbailey@foobar.com|
|1003|    Edward|   Walker|       ed@walker.com|
+----+----------+---------+--------------------+



the following is an example of stream data with upsert, in order to avoid duplication by id
the input is CSV file and the output is bronze delta with only rellevant id's
we might want to change the CSV to Avro, as its more efficient, but less readable
now, execute the insert and update from the README

In [9]:
# Streaming aggregates in Update mode
print("####### Streaming upgrades in update mode ########")

# Function to upsert microBatchOutputDF into Delta Lake table using merge
# if id doesnt exist it inserts if it does, update
def upsertToDelta(microBatchOutputDF, batchId):
    t = deltaTable.alias("t").merge(microBatchOutputDF.alias("s"), "s.id = t.id")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
#schema is taken from files that already exist on the topic 
schema=spark.read.format('json').load(OBJECTURL_TEST).schema
#reading raw data (json format)
streamingRawDF=spark.readStream.format("json").schema(schema).load(OBJECTURL_TEST)
#taking the rellevant data from raw data
streamingAggregatesDF=streamingRawDF.select(col("after.id"), col("after.first_name"), col("after.last_name"), col("after.email"))
# Write the output of a streaming aggregation query into Delta Lake bronze table
#call upsert for each and every new record
stream3 = streamingAggregatesDF.writeStream\
    .format("delta") \
    .foreachBatch(upsertToDelta) \
    .outputMode("update") \
    .option("checkpointLocation", 's3a://minio-sink-bucket/delta/bronze/customers/checkpoints/') \
    .start('s3a://minio-sink-bucket/delta/bronze/customers/data/')
#wait for 10 seconds before continue to stop
stream2 = spark.readStream.format("delta").load('s3a://minio-sink-bucket/delta/bronze/customers/data/')\
    .writeStream\
    .format("console")\
    .start()

stream3.awaitTermination(30)
stream3.stop()
stream2.awaitTermination(300)
stream2.stop()
print("########### DeltaTable after streaming upsert #########")
#deltaTable.toDF().show()

####### Streaming upgrades in update mode ########


21/10/25 08:04:38 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-61d79c58-ce94-40d3-81b7-4f0963f6239f. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+----------+---------+--------------------+
|  id|first_name|last_name|               email|
+----+----------+---------+--------------------+
|1005|      Jane|      Roe|john.doe@example.com|
|1001|     Sally|   Thomas|sally.thomas@acme...|
|1004|      Anne|Kretchmar|  annek@noanswer.org|
|1002|    George|   Bailey|  gbailey@foobar.com|
|1003|    Edward|   Walker|       ed@walker.com|
+----+----------+---------+--------------------+



21/10/25 08:04:50 ERROR MicroBatchExecution: Query [id = d8fa7911-048e-4eaf-a5c0-2a4edc369d32, runId = eac2a6f0-12a6-4624-8352-bccf104a60e0] terminated with error
java.lang.UnsupportedOperationException: Detected a data update (for example part-00000-cf1340f7-5a58-4d11-8e8a-8c93bf810d9d-c000.snappy.parquet) in the source table at version 3. This is currently not supported. If you'd like to ignore updates, set the option 'ignoreChanges' to 'true'. If you would like the data update to be reflected, please restart this query with a fresh checkpoint directory.
	at org.apache.spark.sql.delta.DeltaErrors$.deltaSourceIgnoreChangesError(DeltaErrors.scala:137)
	at org.apache.spark.sql.delta.sources.DeltaSource.verifyStreamHygieneAndFilterAddFiles(DeltaSource.scala:348)
	at org.apache.spark.sql.delta.sources.DeltaSource.$anonfun$getFileChanges$1(DeltaSource.scala:191)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:484)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:

StreamingQueryException: Detected a data update (for example part-00000-cf1340f7-5a58-4d11-8e8a-8c93bf810d9d-c000.snappy.parquet) in the source table at version 3. This is currently not supported. If you'd like to ignore updates, set the option 'ignoreChanges' to 'true'. If you would like the data update to be reflected, please restart this query with a fresh checkpoint directory.
=== Streaming Query ===
Identifier: [id = d8fa7911-048e-4eaf-a5c0-2a4edc369d32, runId = eac2a6f0-12a6-4624-8352-bccf104a60e0]
Current Committed Offsets: {DeltaSource[s3a://minio-sink-bucket/delta/bronze/customers/data]: {"sourceVersion":1,"reservoirId":"2c0b8639-0409-4cb5-ac04-4c57070182db","reservoirVersion":2,"index":4,"isStartingVersion":true}}
Current Available Offsets: {DeltaSource[s3a://minio-sink-bucket/delta/bronze/customers/data]: {"sourceVersion":1,"reservoirId":"2c0b8639-0409-4cb5-ac04-4c57070182db","reservoirVersion":2,"index":4,"isStartingVersion":true}}

Current State: ACTIVE
Thread State: RUNNA

In [ ]:
deltaTable.toDF().show()

example for updating according to input event

In [ ]:
# Streaming aggregates in Update mode
print("####### Streaming upgrades in update mode ########")

# Function to upsert microBatchOutputDF into Delta Lake table using merge
# if id doesnt exist it inserts if it does, update
def upsertToDeltaByBefore(microBatchOutputDF, batchId):
    t = deltaTable.alias("t").merge(microBatchOutputDF.alias("s"), 's.id = t.id')\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

#raw data received from kafka is stored under S3 customers
OBJECTURL_TEST = 's3a://minio-sink-bucket/topics/customers'
#schema is taken from files that already exist on the topic 
schema=spark.read.format('json').load(OBJECTURL_TEST).schema
#reading raw data (json format)
streamingRawDF=spark.readStream.format("json").schema(schema).load(OBJECTURL_TEST)
#taking the rellevant data from raw data
streamingAggregatesDF=streamingRawDF.select(col('before'), col("after.id"), col("after.first_name"), col("after.last_name"), col("after.email"))
# Write the output of a streaming aggregation query into Delta Lake bronze table
deltaTable = DeltaTable.forPath(spark, 's3a://minio-sink-bucket/delta/bronze/customers/data/')
print("#############  Original Delta Table ###############")
deltaTable.toDF().show()
#call upsert for each and every new record
stream3 = streamingAggregatesDF.writeStream\
    .format("delta") \
    .foreachBatch(upsertToDeltaByBefore) \
    .outputMode("update") \
    .start()
#wait for 100 seconds before continue to stop
stream3.awaitTermination(100)
stream3.stop()
print("########### DeltaTable after streaming upsert #########")
deltaTable.toDF().show()

In [ ]:
spark.read.format("json").schema(schema).load(OBJECTURL_TEST).collect()

In [ ]:
deltaTable.delete()

In [ ]:
deltaTable.toDF().show()

In [ ]:
deltaTable.vacuum()  

In [ ]:
spark.range(numRows).collect()

In [ ]:
# Streaming append and concurrent repartition using  data change = false
# tbl1 is the sink and tbl2 is the source
print("############ Streaming appends with concurrent table repartition  ##########")
tbl1 = 's3a://minio-sink-bucket/test/delta-table4'
tbl2 = "s3a://minio-sink-bucket/test/delta-table5"
numRows = 10
spark.range(numRows).write.mode("overwrite").format("delta").save(tbl1)
spark.read.format("delta").load(tbl1).show()
spark.range(numRows, numRows * 10).write.mode("overwrite").format("delta").save(tbl2)


In [ ]:
# Start reading tbl2 as a stream and do a streaming write to tbl1
# Prior to Delta 0.5.0 this would throw StreamingQueryException: Detected a data update in the
# source table. This is currently not supported.
stream4 = spark.readStream.format("delta").load(tbl2).writeStream.format("delta")\
    .option("checkpointLocation", "s3a://minio-sink-bucket/test/checkpoint/tbl1") \
    .outputMode("append") \
    .start(tbl1)

# repartition table while streaming job is running
spark.read.format("delta").load(tbl2).repartition(10).write\
    .format("delta")\
    .mode("overwrite")\
    .option("dataChange", "false")\
    .save(tbl2)

stream4.awaitTermination(10)
stream4.stop()
print("######### After streaming write #########")

In [ ]:
spark.read.format("delta").load(tbl1).show()


In [ ]:
import time
OBJECTURL_TEST = 's3a://minio-sink-bucket/topics/customers'

# Read the data back from MinIO
gnames_df = spark.read.format('json') \
    .load(OBJECTURL_TEST)
gnames_df.show()

In [ ]:
schema=spark.read.format('json').load(OBJECTURL_TEST).schema
streamingDf=spark.readStream.format("json").schema(schema).load(OBJECTURL_TEST)
stream=sdf.select(col("after.id"), col("after.first_name"), col("after.last_name"), col("after.email")).writeStream \
  .format("delta") \
  .outputMode("append") \
  .option("checkpointLocation", 's3a://minio-sink-bucket/delta/bronze/customers/checkpoints/').start('s3a://minio-sink-bucket/delta/bronze/customers/data/')
stream.awaitTermination(10)
stream.stop()

In [ ]:




stream3 = streamingAggregatesDF.writeStream\
    .format("delta") \
    .foreachBatch(upsertToDelta) \
    .outputMode("update") \
    .start()
stream3.awaitTermination(10)
stream3.stop()
print("########### DeltaTable after streaming upsert #########")
deltaTable.toDF().show()

In [ ]:
# Stream reads from a table
print("##### Reading from stream ######")
stream2 = spark.readStream.format("delta").load('s3a://minio-sink-bucket/delta/bronze/customers/data/')\
    .writeStream\
    .format("console")\
    .start()
stream2.awaitTermination()
#stream2.stop()

In [ ]:
deltaTable = DeltaTable.forPath(spark, "s3a://minio-sink-bucket/delta/bronze/customers/data/")
print("#############  Original Delta Table ###############")
deltaTable.toDF().show()

In [ ]:
streamingAggregatesDF = spark.readStream.format("s3a://minio-sink-bucket/delta/bronze/customers/data/")

In [ ]:
stream3 = streamingAggregatesDF.writeStream\
    .format("delta") \
    .foreachBatch(upsertToDelta) \
    .outputMode("update") \
    .start()
stream3.awaitTermination(10)
stream3.stop()
print("########### DeltaTable after streaming upsert #########")
deltaTable.toDF().show()

In [ ]:
# Streaming aggregates in Update mode
print("####### Streaming upgrades in update mode ########")

# Function to upsert microBatchOutputDF into Delta Lake table using merge
def upsertToDelta(microBatchOutputDF, batchId):
    t = deltaTable.alias("t").merge(microBatchOutputDF.alias("s"), "s.id = t.id")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

In [ ]:
# Write the output of a streaming aggregation query into Delta Lake table
deltaTable = DeltaTable.forPath(spark, "/tmp/delta-streaming/delta-table")
print("#############  Original Delta Table ###############")
deltaTable.toDF().show()
stream3 = streamingAggregatesDF.writeStream\
    .format("delta") \
    .foreachBatch(upsertToDelta) \
    .outputMode("update") \
    .start()
stream3.awaitTermination(10)
stream3.stop()

In [ ]:
spark.read.format("delta").load('s3a://minio-sink-bucket/delta/bronze/customers/data/').collect()

In [ ]:
 df = spark.readStream.format("cloudFiles")\
   .option('cloudFiles.format', 'text')\
   .load('/mnt/tlogs/staging')
 # updated to remove EntryMethod which apear twice and not allowed by delta lake
 
 bronze=df.select(from_json(regexp_replace('value', 'EntryMethod', 'rabEntryM'),schema_s).alias('value')).\
           select(col('value._id').alias("tlogId"),col('value.organization').alias("siteId"),to_date('value.businessDate').\
           alias("businessDate"),explode('value.order').alias('order'))\
           .withColumn('orderType', col('order.orderType')).withColumn('orderTotalAmount', col('order.totals.totalAmount'))\
           .withColumn('orderId',col('order._id')).writeStream.partitionBy('siteId').format("delta").\
            option("checkpointLocation", '/mnt/tlogs/checkpoints/bronze_cp').option('multiline', True).start('/mnt/tlogs/deltas/bronze')

In [ ]:
from pyspark.sql.functions import from_json, col
gnames_df.select(col("after.id"), col("after.first_name"), col("after.last_name"), col("after.email")).collect()

In [ ]:
!pip install koalas==1.8.1

In [ ]:
from databricks import koalas as ks
kdf = gnames_df.to_koalas()

In [ ]:
kdf['after'][0]

In [ ]:
table_dir = "s3a://test-container/playground/delta-table"
spark.sql("CREATE TABLE delta.`%s`(id LONG) USING delta" % table_dir)


In [ ]:
spark.sql("VACUUM '%s' RETAIN 169 HOURS" % table_dir).collect()

In [ ]:
import random

data = spark.range(8)
data = data.withColumn("value", data.id + random.randint(0, 5000))
data.take(10)

In [ ]:
spark.sql("INSERT INTO delta.`%s` VALUES 0, 1, 2, 3, 4" % table_dir)

In [ ]:
spark.sql("SELECT * FROM delta.`%s`" % table_dir).show()

In [ ]:
!pip install koalas==1.8.1

In [ ]:
import time
OBJECTURL_TEST = 's3a://test-container/playground/colors-test' + str(time.time()) + '.csv'
rdd = sc.parallelize([('Mario', 'Red'), ('Luigi', 'Green'), ('Princess', 'Pink')])
rdd.toDF(['name', 'color']).write.csv(OBJECTURL_TEST, header=True)

# Read the data back from MinIO
gnames_df = spark.read.format('csv').option('header', True) \
    .load(OBJECTURL_TEST)
gnames_df.show()

In [ ]:
DELTA_URL='s3a://test-container/playground/delts-colors-test' + str(time.time())
gnames_df.write.format("delta").save(DELTA_URL)

In [ ]:
data = spark.range(0, 5)
#data.write.format("delta").save("/tmp/delta-table")

In [ ]:
data.write.format("delta").save("s3a://test-container/playground/to_overide")

In [ ]:
data.write.format("delta").mode("overwrite").save("s3a://test-container/playground/to_overide")

In [ ]:
from databricks import koalas as ks
import pandas as pd
import numpy as np

In [ ]:
s = ks.Series([1, 3, 5, np.nan, 6, 8])

In [ ]:
kdf = ks.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [ ]:
kdf

In [ ]:
dates = pd.date_range('20130101', periods=6)
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))
kdf = ks.from_pandas(pdf)
print(type(kdf))
kdf

In [ ]:
sdf = spark.createDataFrame(pdf)
sdf.show()

In [ ]:
kdf = sdf.to_koalas()

In [ ]:
sdf.collect()

In [ ]:
kdf.to_parquet("s3a://test-container/playground/kdf")

In [ ]:
ks.read_parquet("s3a://test-container/playground/kdf").head(10)

In [ ]:
kdf.to_delta("s3a://test-container/playground/delta_partitioned", mode='overwrite', partition_cols=["A","B"])

In [ ]:
ks.read_delta("s3a://test-container/playground/delta").head(10)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from delta.tables import DeltaTable
import shutil
import random

In [ ]:
data = spark.range(8)
data = data.withColumn("value", data.id + random.randint(0, 5000))
data.write.format("delta").mode("overwrite").save("s3a://test-container/playground/delta-streaming/delta-table")

In [ ]:
# Stream writes to the table
print("####### Streaming write ######")
streamingDf = spark.readStream.format("rate").load()
stream = streamingDf.selectExpr("value as id").writeStream\
    .format("delta")\
    .option("checkpointLocation", "s3a://test-container/playground/delta-streaming/checkpoint")\
    .start("s3a://test-container/playground/delta-streaming/delta-table2")
stream.awaitTermination(10)
stream.stop()

In [ ]:
spark.sql("SELECT * FROM delta.`%s`" % "s3a://test-container/playground/delta-streaming/delta-table2").show()

In [ ]:
# Stream reads from a table
print("##### Reading from stream ######")
stream2 = spark.readStream.format("delta").load("s3a://test-container/playground/delta-streaming/delta-table2")\
    .writeStream\
    .format("console")\
    .start()
stream2.awaitTermination(10)
stream2.stop()

In [ ]:
print("####### Streaming upgrades in update mode ########")

In [ ]:
# Function to upsert microBatchOutputDF into Delta Lake table using merge
def upsertToDelta(microBatchOutputDF, batchId):
    t = deltaTable.alias("t").merge(microBatchOutputDF.alias("s"), "s.id = t.id")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()

In [ ]:
from pyspark.sql.functions import *


streamingAggregatesDF = spark.readStream.format("rate").load()\
    .withColumn("id", col("value") % 10)\
    .drop("timestamp")

# Write the output of a streaming aggregation query into Delta Lake table
deltaTable = DeltaTable.forPath(spark, "s3a://test-container/playground/delta-streaming/delta-table")
print("#############  Original Delta Table ###############")
deltaTable.toDF().show()


In [ ]:
stream3 = streamingAggregatesDF.writeStream\
    .format("delta") \
    .foreachBatch(upsertToDelta) \
    .outputMode("update") \
    .start()
stream3.awaitTermination(10)
stream3.stop()
print("########### DeltaTable after streaming upsert #########")

In [ ]:
streamingAggregatesDF = spark.readStream.format("rate").load()\
    .withColumn("id", col("value") % 10)\
    .drop("timestamp")
# Write the output of a streaming aggregation query into Delta Lake table
deltaTable = DeltaTable.forPath(spark, "s3a://test-container/playground/delta-streaming/delta-table")
print("#############  Original Delta Table ###############")
deltaTable.toDF().show()
stream3 = streamingAggregatesDF.writeStream\
    .format("delta") \
    .foreachBatch(upsertToDelta) \
    .outputMode("update") \
    .start()
stream3.awaitTermination(10)
stream3.stop()
print("########### DeltaTable after streaming upsert #########")
deltaTable.toDF().show()

# Streaming append and concurrent repartition using  data change = false
# tbl1 is the sink and tbl2 is the source
print("############ Streaming appends with concurrent table repartition  ##########")
tbl1 = "s3a://test-container/playground/delta-streaming/delta-table4"
tbl2 = "s3a://test-container/playground/delta-streaming/delta-table5"
numRows = 10
spark.range(numRows).write.mode("overwrite").format("delta").save(tbl1)
spark.read.format("delta").load(tbl1).show()
spark.range(numRows, numRows * 10).write.mode("overwrite").format("delta").save(tbl2)


# Start reading tbl2 as a stream and do a streaming write to tbl1
# Prior to Delta 0.5.0 this would throw StreamingQueryException: Detected a data update in the
# source table. This is currently not supported.
stream4 = spark.readStream.format("delta").load(tbl2).writeStream.format("delta")\
    .option("checkpointLocation", "s3a://test-container/playground/delta-streaming/checkpoint/tbl1") \
    .outputMode("append") \
    .start(tbl1)

# repartition table while streaming job is running
spark.read.format("delta").load(tbl2).repartition(10).write\
    .format("delta")\
    .mode("overwrite")\
    .option("dataChange", "false")\
    .save(tbl2)

stream4.awaitTermination(10)
stream4.stop()
print("######### After streaming write #########")
spark.read.format("delta").load(tbl1).show()
# cleanup
try:
    shutil.rmtree("s3a://test-container/playground/delta-streaming/")
except:
    pass

In [ ]:
spark.sql("CREATE TABLE delta.`%s`(id LONG)" % "s3a://test-container/playground/delta-table")